In [1]:
import os
import shutil
from cv2 import *
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

In [66]:
tf.keras.backend.clear_session()


In [5]:
kernel = np.ones((5,5), np.uint8)
train_path = 'chest_xray/train/'
new_train_path = 'New Images/Train'
for files in os.listdir(train_path):

    new_train_path2 = os.path.join(new_train_path,files)

    for f in os.listdir(train_path+files):

        img_path=os.path.join(train_path+files,f)
        norm=imread(img_path)
        norm=cvtColor(norm,COLOR_BGR2GRAY)
        th1=equalizeHist(norm)
        eroded = cv2.erode(th1, kernel)
        dilate = cv2.dilate(eroded,kernel)
        eroded2 = cv2.erode(dilate,kernel)

        new_train_path3 = os.path.join(new_train_path2,f)

        imwrite(new_train_path3,eroded2)

test_path = 'chest_xray/test/'
new_test_path = 'New Images/Test'

for files in os.listdir(test_path):

    new_test_path2 = os.path.join(new_test_path,files)

    for f in os.listdir(test_path+files):

        img_path=os.path.join(test_path+files,f)

        norm=imread(img_path)
        norm=cvtColor(norm,COLOR_BGR2GRAY)
        th1 = equalizeHist(norm)
        eroded = cv2.erode(th1, kernel)
        dilate = cv2.dilate(eroded, kernel)
        eroded2 = cv2.erode(dilate, kernel)

        new_test_path3 = os.path.join(new_test_path2,f)

        imwrite(new_test_path3, eroded2)


In [3]:
IMG_SIZE = 300

TRAINING_DIR = "New Images/Train"
training_datagen = ImageDataGenerator(rescale = 1/255 ,
                                  #     rotation_range=15,
                                  # height_shift_range=0.2,
                                  # width_shift_range=0.2,
                                  # shear_range=0.2,
                                  zoom_range=0.3,
                                  vertical_flip=True,
                                  fill_mode='nearest')
train_generator = training_datagen.flow_from_directory(TRAINING_DIR,
                                                       target_size=(IMG_SIZE,IMG_SIZE) ,class_mode='binary',
                                                       batch_size=64,shuffle=True )


TEST_DIR = "New Images/Test"
test_datagen = ImageDataGenerator(rescale = 1/255)
test_generator = test_datagen.flow_from_directory(TEST_DIR,target_size=(IMG_SIZE,IMG_SIZE), class_mode='binary',
                                                  batch_size=64,
                                                  shuffle=False)

Found 6581 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [4]:
feature_extractor = tf.keras.applications.ResNet50V2(include_top=False,weights='imagenet',input_shape=(IMG_SIZE,IMG_SIZE,3))
feature_extractor.trainable = False

In [5]:
model = tf.keras.models.Sequential([
    feature_extractor,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')

])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50v2 (Functional)      (None, 10, 10, 2048)      23564800  
_________________________________________________________________
flatten (Flatten)            (None, 204800)            0         
_________________________________________________________________
dense (Dense)                (None, 128)               26214528  
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               33024     
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               6

In [67]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self,epoch,logs={}):
    if(logs['accuracy']>=0.975):
      self.model.stop_training=True

callbacks=myCallback()
METRICS = [
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')]

checkpoint = ModelCheckpoint(filepath='best_weights.hdf5', save_best_only=True, save_weights_only=True)
lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=2, mode='max')

# early_stop = EarlyStopping(monitor='val_loss', min_delta=0.1, patience=3, mode='min')

model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.0001),loss='binary_crossentropy',metrics=METRICS )

history = model.fit(train_generator , epochs=20 , callbacks=[callbacks,checkpoint,lr_reduce], validation_data=test_generator)

Epoch 1/20
103/103 [==============================] - 162s 2s/step - loss: 0.1246 - accuracy: 0.9546 - precision: 0.9650 - recall: 0.9578 - val_loss: 0.1022 - val_accuracy: 0.9615 - val_precision: 0.9621 - val_recall: 0.9769
Epoch 2/20
103/103 [==============================] - 155s 2s/step - loss: 0.1310 - accuracy: 0.9511 - precision: 0.9614 - recall: 0.9554 - val_loss: 0.1201 - val_accuracy: 0.9567 - val_precision: 0.9595 - val_recall: 0.9718
Epoch 3/20
103/103 [==============================] - 157s 2s/step - loss: 0.1138 - accuracy: 0.9591 - precision: 0.9696 - recall: 0.9609 - val_loss: 0.0975 - val_accuracy: 0.9728 - val_precision: 0.9722 - val_recall: 0.9846
Epoch 4/20
103/103 [==============================] - 156s 2s/step - loss: 0.1062 - accuracy: 0.9623 - precision: 0.9720 - recall: 0.9639 - val_loss: 0.1121 - val_accuracy: 0.9615 - val_precision: 0.9716 - val_recall: 0.9667
Epoch 5/20
103/103 [==============================] - ETA: 0s - loss: 0.0950 - accuracy: 0.9638 - pr

In [68]:
model.evaluate(test_generator, batch_size=32)

10/10 [==============================] - 5s 509ms/step - loss: 0.1115 - accuracy: 0.9599 - precision: 0.9692 - recall: 0.9667


[0.11149944365024567,
 0.9599359035491943,
 0.9691516757011414,
 0.9666666388511658]

In [74]:
model.save('model_with_975_early_stopping.h5')